# My Comic Book

### DIY comic book using Generative AI
#### Create your comic with just a single prompt

In [ ]:
# !apt-get update
# !apt-get install -y libreoffice
# !pip install openai tiktoken langchain replicate kor python-docx unoconv

## Setup environment variables

In [ ]:
import os
os.environ["REPLICATE_API_TOKEN"] = "PASTE_TOKEN_HERE"
os.environ["OPENAI_API_KEY"] = "PASTE_TOKEN_HERE"
os.environ["OPENAI_ORGANIZATION"] = "PASTE_ORG_ID__HERE"

In [ ]:
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO
import cv2
from google.colab.patches import cv2_imshow

from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT

import subprocess

from langchain import PromptTemplate, LLMChain
from langchain.llms import Replicate, OpenAI

## Create Prompt Template

In [ ]:
template = """Write a short story about {topic}.
First generate a title for the story and a cover picture. Make the story in a narrative format.
Divide it into 5 chapters. Limit each chapter to 200 words. Also generate a prompt for an image generation model at the end of each chapter.
Always generate the output in following format:

Title: Main title of the story
Cover Image: Prompt for image generation model
Chapter 1:
Title: Title of chapter 1
Text: Story text of chapter 1
Image Prompt: Prompt for image generation model
Chapter 2:
Title: Title of chapter 2
Text: Story text of chapter 2
Image Prompt: Prompt for image generation model
Chapter n:
Title: Title of chapter n
Text: Story text of chapter n
Image Prompt: Prompt for image generation model



"""

prompt = PromptTemplate(template=(template), input_variables=["topic"])

## Setup Chain

In [ ]:
llm = OpenAI(temperature=0.7, model="text-davinci-003", max_tokens=2000)
llm_chain = LLMChain(prompt=prompt, llm=llm)

## Generate story based on topic

In [ ]:
## ENTER YOUR TOPIC HERE
topic_text = input()

result = llm_chain.run(
    topic_text
)

In [ ]:
print(result)

## Save the result into a text file

In [ ]:
with open("story.txt", 'w+') as file:
    file.write(result)

## Convert the text into structured (dictionary) format to feed to downstream tasks

In [ ]:
with open('story.txt', 'r') as file:
    lines = file.readlines()

story = {}
is_chapter = False

chapters = []
chapter_dict = {}

for line in lines:



    line_sm = line.lower().strip()

    if 'title' in line_sm and not is_chapter:
        story['title'] = line.split(':')[-1].strip()

    if 'title' in line_sm and is_chapter:
        chapter_dict['title'] = line.split(':')[-1].strip()

    if 'cover image' in line_sm:
        story['cover_image'] = line.split(':')[-1].strip()

    if 'text' in line_sm:
        chapter_dict['text'] = line.split(':')[-1].strip()

    if 'image prompt' in line_sm:
        chapter_dict['image_prompt'] = line.split(':')[-1].strip()
        chapters.append(chapter_dict)

    if 'chapter' in line_sm:
        is_chapter = True
        chapter_dict = {}
        continue

story['chapters'] = chapters
story

In [ ]:
image_prompts_dict = {"cover_image": story["cover_image"]}
image_prompts_dict

In [ ]:
chapter_image_prompts = []
for chapter in story["chapters"]:
    if "image_prompt" in chapter:
        chapter_image_prompts.append(chapter["image_prompt"])

image_prompts_dict["chapter_image_prompts"] = chapter_image_prompts
image_prompts_dict

### Setup stable diffusion

In [ ]:
text2image = Replicate(
    model="stability-ai/stable-diffusion:db21e45d3f7023abc2a46ee38a23973f6dce16bb082a930b0c49861f96d1e5bf",
    input={"image_dimensions": "512x512"},
)

## Generate and save images

In [ ]:
generated_images_dict = {}

In [ ]:
img_path = './images/'

if not os.path.exists(img_path):
    os.makedirs(img_path)

In [ ]:
style_prompts = " Graphic Novel, 4K, Global Illumination, Dreamy"

In [ ]:
cover_image_prompt = image_prompts_dict["cover_image"]


# style of image
cover_image_prompt += style_prompts

# execute image generation pipeline
image_output = text2image(cover_image_prompt)

response = requests.get(image_output)
image_output = Image.open(BytesIO(response.content))

cover_path = os.path.join(img_path, 'cover_image.jpg')
image_output.save(cover_path)

generated_images_dict["cover_image"] = cover_path

In [ ]:
chapter_images = []
for idx, chapter_image_prompts in tqdm(enumerate(image_prompts_dict["chapter_image_prompts"])):
    chapter_image_prompts += style_prompts
    image_output = text2image(chapter_image_prompts)
    response = requests.get(image_output)
    image_output = Image.open(BytesIO(response.content))
    chapter_path = os.path.join(img_path, f'chapter_{idx+1}.jpg')
    image_output.save(chapter_path)
    chapter_images.append(chapter_path)

generated_images_dict["chapter_images"] = chapter_images

In [ ]:
generated_images_dict

In [ ]:

# List of image and text combinations
image_text_combinations = []
chapters = story['chapters']

for idx, img_path in enumerate(generated_images_dict['chapter_images']):
    chapter_dict = chapters[idx]
    story_text = chapter_dict['title'] + '\n\n' + chapter_dict['text']
    image_text_combinations.append((img_path, story_text))

## Put image and text into docx file

In [ ]:

# Create a new Word document
doc = Document()

title = story['title']
title_paragraph = doc.add_paragraph(title)
title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
title_paragraph.runs[0].bold = True
title_paragraph.runs[0].font.size = Pt(18)
doc.add_picture(generated_images_dict['cover_image'], width=Inches(4), height=Inches(3))


# Loop through each combination and add to the document
for image_filename, long_text in image_text_combinations:
    doc.add_page_break()  # Add a page break for each combination

    # Create a table with 1 row and 2 columns
    table = doc.add_table(rows=1, cols=2)
    table.autofit = False
    table.columns[0].width = Inches(4)  # Adjust the width of the first column
    table.columns[1].width = Inches(2)  # Adjust the width of the second column

    # Add the image to the first cell
    cell_1 = table.cell(0, 0)
    image = cell_1.add_paragraph().add_run()
    image.add_picture(image_filename, width=Inches(4), height=Inches(3))  # Adjust width as needed

    # Add the text to the second cell
    cell_2 = table.cell(0, 1)
    paragraph = cell_2.add_paragraph(long_text)
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

# Save the Word document
doc.save("output.docx")


## Convert docx to pdf

In [ ]:
import subprocess

input_docx_path = "output.docx"  # Replace with your actual input path
output_pdf_path = "output.pdf"  # Replace with your desired output path

try:
    subprocess.run(["unoconv", "-f", "pdf", "-o", output_pdf_path, input_docx_path], check=True)
    print("Conversion successful!")
except subprocess.CalledProcessError as e:
    print("Error:", e)
